# NLP (comentários página do Facebook)
Este notebook contém o código referente ao modelo que analisa sentimentos em formato textual. A proposta aqui foi a de criar um modelo que pudesse analisar os comentários públicos da página do Facebook da Passos Mágicos e assim demonstrar o quão impactante suas ações são nas vidas das pessoas. Foram considerados 3 categorias de comentários: positivo, negativo e neutro.

Aqui vale notar que além dos comentários da página do Facebook da ONG, também foram gerados batches de comentários pelo ChatGPT da OpenAI. Isso se fez necessário, pois não haviam muitos comentários ou reviews da ONG que pudessem ser utilizados para treinar efetivamente um NLP.

Os comentários e reviews originais da página da ONG foram misturados juntos dos comentários gerados pelo ChatGPT.

In [77]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [180]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import unicodedata
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import svm
import joblib

In [107]:
pd.set_option('display.max_columns', None)

In [79]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [80]:
print(spacy.util.get_installed_models())

['pt_core_news_sm', 'en_core_web_sm']


In [81]:
nlp_spacy_ptbr = spacy.load("pt_core_news_sm")

## Funções utilitárias para limpeza e normalização de strings

In [82]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

def lemmatize(text):
    doc = nlp_spacy_ptbr(text)
    return " ".join([token.lemma_ for token in doc])

## Carregando os comentários e reviews da página do Facebook da ONG Passos Mágicos + comentários gerados pelo ChatGPT da OpenAI

In [83]:
df_1 = pd.read_csv('/content/chatgpt-base-comentarios-negativos.csv', sep=';')
df_2 = pd.read_csv('/content/chatgpt-base-comentarios-neutros.csv', sep=';')
df_3 = pd.read_csv('/content/chatgpt-base-comentarios-positivos.csv', sep=';')
df_4 = pd.read_csv('/content/facebook-mentions.csv', sep=';')
df_5 = pd.read_csv('/content/facebook-reviews.csv', sep=';')
df = pd.concat([df_1, df_2, df_3, df_4, df_5])
df

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo
...,...,...
7,De projeto...uma ONG! Exemplo para muitas pess...,positivo
8,Projeto sério realizado por uma equipe extrema...,positivo
9,“A educação é a arma mais poderosa que você po...,positivo
10,UM exemplo de instituição... estão de parabéns...,positivo


In [84]:
df['categoria'].unique()

array(['negativo', 'neutro', 'positivo'], dtype=object)

In [85]:
df.shape

(1060, 2)

In [86]:
df.head()

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo


In [87]:
df = shuffle(df)
df = df.reset_index(drop=True)

In [88]:
df.head()

,texto,categoria
0,Produto em bom estado.,neutro
1,A comunicação da ONG é ruim e os doadores fica...,negativo
2,Recebi o tamanho correto.,positivo
3,Produto novo e funcional.,neutro
4,Produto muito inferior.,negativo


In [89]:
df['texto_processado'] = df['texto'].apply(tokenizer)
df['texto_processado_lemm'] = df['texto_processado'].apply(lemmatize)
df.head()

,texto,categoria,texto_processado,texto_processado_lemm
0,Produto em bom estado.,neutro,produto bom estado,produto bom estado
1,A comunicação da ONG é ruim e os doadores fica...,negativo,comunicacao ong ruim doadores ficam informacoe...,comunicacao ong ruim doador ficar informacoe c...
2,Recebi o tamanho correto.,positivo,recebi tamanho correto,recebir tamanho correto
3,Produto novo e funcional.,neutro,produto novo funcional,produto novo funcional
4,Produto muito inferior.,negativo,produto inferior,produto inferior


## Dividindo o dataset em treino e teste

In [90]:
X = df['texto_processado'].values
y = df['categoria'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

In [91]:
print('X_train:', X_train.size)
print('X_test:', X_test.size)
print('y_train:', y_train.size)
print('y_test:', y_test.size)

X_train: 848
X_test: 212
y_train: 848
y_test: 212


In [92]:
vect = CountVectorizer(lowercase=False)
vect.fit(X_train)

CountVectorizer(lowercase=False)

In [93]:
X_train = vect.transform(X_train)
X_train

<848x709 sparse matrix of type '<class 'numpy.int64'>'
	with 4278 stored elements in Compressed Sparse Row format>

In [94]:
X_test = vect.transform(X_test)
X_test

<212x709 sparse matrix of type '<class 'numpy.int64'>'
	with 965 stored elements in Compressed Sparse Row format>

## Utilização do modelo SVM para categorizar as frases

In [95]:
modelo = svm.SVC(kernel='linear')
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

In [96]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7735849056603774


## Teste do modelo

In [97]:
texto_teste = tokenizer('Não gostei da ONG, não prestaram o serviço de forma adequada.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['negativo'], dtype=object)

In [98]:
texto_teste = tokenizer('Gostei da ONG!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['positivo'], dtype=object)

In [99]:
texto_teste = tokenizer('A OGN poderia oferecer mais serviços, mas ok.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

In [100]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

## Exportando o modelo para uso no Streamlit

In [101]:
arquivo_modelo = "model.pkl"
arquivo_vect = "vect.pkl"

joblib.dump(modelo, arquivo_modelo)
joblib.dump(vect, arquivo_vect)

['vect.pkl']

## Importando o modelo para teste

In [102]:
modelo = joblib.load(arquivo_modelo)
vect = joblib.load(arquivo_vect)

In [103]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

In [104]:
texto_teste = tokenizer('ONG maravilhosa')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['negativo'], dtype=object)

# Utilizando NLP para identificar o teor dos comentários dos professores e pedagogos da Passos Mágicos

In [108]:
df = pd.read_csv('/content/processado_base_full.csv', sep=';')
df.head()

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
0,Escola Pública,ALUNO-1,11.0,2.0,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Escola Pública,ALUNO-3,12.0,2.0,3H,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Escola Pública,ALUNO-4,10.0,2.0,1D,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Escola Pública,ALUNO-5,10.0,1.0,2M,Não,8.077085,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
df_comentarios_destaque_ieg = df[df['DESTAQUE_IEG'].notna()]['DESTAQUE_IEG'].unique()
df_comentarios_destaque_ieg

array(['Seu destaque em 2020: A sua boa entrega das lições de casa.',
       'Ponto a melhorar em 2021: Melhorar a sua entrega de lições de casa.',
       'Seu destaque em 2020: O seu bom engajamento nas ações de voluntariado.',
       'Ponto a melhorar em 2021: Melhorar o seu engajamento nas ações de voluntariado.',
       'Destaque: A sua boa entrega das lições de casa.',
       'Melhorar: Melhorar a sua entrega de lições de casa.'],
      dtype=object)

In [167]:
df_comentarios_destaque_ida = df[df['DESTAQUE_IDA'].notna()]['DESTAQUE_IDA'].unique()
df_comentarios_destaque_ida

array(['Ponto a melhorar em 2021: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: As suas boas notas na Passos Mágicos.',
       'Seu destaque em 2020: As suas boas notas na Faculdade.',
       'Ponto a melhorar em 2021: As suas notas na Faculdade.',
       'Destaque: As suas boas notas na Passos Mágicos.',
       'Melhorar: Empenhar-se mais nas aulas e avaliações.'], dtype=object)

In [168]:
df_comentarios_destaque_ipv = df[df['DESTAQUE_IPV'].notna()]['DESTAQUE_IPV'].unique()
df_comentarios_destaque_ipv

array(['Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.',
       'Ponto a melhorar em 2021: Integrar-se mais aos Princípios Passos Mágicos.',
       'Destaque: A sua boa integração aos Princípios Passos Mágicos.',
       'Melhorar: Integrar-se mais aos Princípios Passos Mágicos.'],
      dtype=object)

In [169]:
df_comentarios_rec_equipe1 = df[df['REC_EQUIPE_1'].notna()]['REC_EQUIPE_1'].unique()
df_comentarios_rec_equipe1

array(['Promovido de Fase', 'Promovido de Fase + Bolsa', 'Não avaliado',
       'Mantido na Fase atual', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [170]:
df_comentarios_rec_equipe2 = df[df['REC_EQUIPE_2'].notna()]['REC_EQUIPE_2'].unique()
df_comentarios_rec_equipe2

array(['Promovido de Fase', 'Mantido na Fase atual', 'Não avaliado',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [171]:
df_comentarios_rec_equipe3 = df[df['REC_EQUIPE_3'].notna()]['REC_EQUIPE_3'].unique()
df_comentarios_rec_equipe3

array(['Mantido na Fase atual', 'Promovido de Fase', 'Não avaliado',
       'Mantido na Fase + Bolsa', 'Promovido de Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [172]:
df_comentarios_rec_equipe4 = df[df['REC_EQUIPE_4'].notna()]['REC_EQUIPE_4'].unique()
df_comentarios_rec_equipe4

array(['Não avaliado', 'Mantido na Fase atual', 'Promovido de Fase',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [173]:
df_comentarios_rec_ava1 = df[df['REC_AVA_1'].notna()]['REC_AVA_1'].unique()
df_comentarios_rec_ava1

array(['Mantido na Fase + Bolsa', 'Promovido de Fase',
       'Mantido na Fase atual', 'Promovido de Fase + Bolsa',
       'Não avaliado', 'Alocado em Fase anterior'], dtype=object)

In [174]:
df_comentarios_rec_ava2 = df[df['REC_AVA_2'].notna()]['REC_AVA_2'].unique()
df_comentarios_rec_ava2

array(['Mantido na Fase atual', 'Promovido de Fase', 'Não avaliado',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa'],
      dtype=object)

In [175]:
df_comentarios_rec_ava3 = df[df['REC_AVA_3'].notna()]['REC_AVA_3'].unique()
df_comentarios_rec_ava3

array(['Promovido de Fase', 'Mantido na Fase atual',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [176]:
df_comentarios_rec_ava4 = df[df['REC_AVA_4'].notna()]['REC_AVA_4'].unique()
df_comentarios_rec_ava4

array(['Promovido de Fase', 'Promovido de Fase + Bolsa',
       'Mantido na Fase atual', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [186]:
array_nlp_processamento = np.concatenate((df_comentarios_destaque_ieg, df_comentarios_destaque_ida, df_comentarios_destaque_ipv, df_comentarios_rec_equipe1, df_comentarios_rec_equipe2, df_comentarios_rec_equipe3, df_comentarios_rec_equipe4, df_comentarios_rec_ava1, df_comentarios_rec_ava2, df_comentarios_rec_ava3, df_comentarios_rec_ava4))
array_nlp_processamento

array(['Seu destaque em 2020: A sua boa entrega das lições de casa.',
       'Ponto a melhorar em 2021: Melhorar a sua entrega de lições de casa.',
       'Seu destaque em 2020: O seu bom engajamento nas ações de voluntariado.',
       'Ponto a melhorar em 2021: Melhorar o seu engajamento nas ações de voluntariado.',
       'Destaque: A sua boa entrega das lições de casa.',
       'Melhorar: Melhorar a sua entrega de lições de casa.',
       'Ponto a melhorar em 2021: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: As suas boas notas na Passos Mágicos.',
       'Seu destaque em 2020: As suas boas notas na Faculdade.',
       'Ponto a melhorar em 2021: As suas notas na Faculdade.',
       'Destaque: As suas boas notas na Passos Mágicos.',
       'Melhorar: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.',
       'Ponto a melhorar em 2021: Integrar-se mais aos Princípios Passos Mágicos.'

In [188]:
df_validacao_nlp = pd.DataFrame({
    'texto': array_nlp_processamento
})
df_validacao_nlp['tokenizado'] = df_validacao_nlp['texto'].apply(tokenizer)
df_validacao_nlp.head()

,texto,tokenizado
0,Seu destaque em 2020: A sua boa entrega das li...,destaque boa entrega licoes casa
1,Ponto a melhorar em 2021: Melhorar a sua entre...,ponto melhorar melhorar entrega licoes casa
2,Seu destaque em 2020: O seu bom engajamento na...,destaque bom engajamento acoes voluntariado
3,Ponto a melhorar em 2021: Melhorar o seu engaj...,ponto melhorar melhorar engajamento acoes volu...
4,Destaque: A sua boa entrega das lições de casa.,destaque boa entrega licoes casa


In [192]:
df_validacao_nlp['previsao'] = df_validacao_nlp['tokenizado'].apply(lambda x: modelo.predict(vect.transform([x])))
df_validacao_nlp.head(50)

,texto,tokenizado,previsao
0,Seu destaque em 2020: A sua boa entrega das li...,destaque boa entrega licoes casa,[neutro]
1,Ponto a melhorar em 2021: Melhorar a sua entre...,ponto melhorar melhorar entrega licoes casa,[neutro]
2,Seu destaque em 2020: O seu bom engajamento na...,destaque bom engajamento acoes voluntariado,[neutro]
3,Ponto a melhorar em 2021: Melhorar o seu engaj...,ponto melhorar melhorar engajamento acoes volu...,[neutro]
4,Destaque: A sua boa entrega das lições de casa.,destaque boa entrega licoes casa,[neutro]
5,Melhorar: Melhorar a sua entrega de lições de ...,melhorar melhorar entrega licoes casa,[neutro]
6,Ponto a melhorar em 2021: Empenhar-se mais nas...,ponto melhorar empenhar aulas avaliacoes,[neutro]
7,Seu destaque em 2020: As suas boas notas na Pa...,destaque boas notas passos magicos,[positivo]
8,Seu destaque em 2020: As suas boas notas na Fa...,destaque boas notas faculdade,[neutro]
9,Ponto a melhorar em 2021: As suas notas na Fac...,ponto melhorar notas faculdade,[neutro]
